### Presentation (20 marks)

Your project will also be assessed on the basis of a 15-minute video presentation that you must upload to MS Stream before the deadline of **Friday 15th January, 4:00 pm UTC**.

You can record the presentation in any software that you like, but we recommend recording in MS Teams as this allows for simple uploading to MS Stream.

You presentation should provide the following information:

1. A brief description of your airburst solver solution algorithm, including ODE solving routine.
2. A quantification of the accuracy of your numerical solution for two cases, with and without fragmentation, for User-specified input parameters. It is up to you to design an appropriate demonstration of accuracy, but this could take the form of a plot of error vs. timestep size or error vs. solver tolerance, depending on your solution algorithm. 
3. A demonstration of how to use your software to determine the impactor parameters (asteroid size & strength) for the Chelyabinsk airburst, by fitting your model solutions of kinetic energy loss per unit height vs. altitude to the inferred energy deposition curve.
4. A brief description of your algorithm for finding postcodes within each damage zone
5. A brief description of your algorithm for accounting for the effect of uncertainty in the input parameters (risk calculation).
6. A demonstration of your software for a specified scenario that will be provided on Friday.


In [2]:
%%html
<style>
div.file{
    display: block;
    background-color: #D3D3D3;
    border-color: #8A2BE2;
    border-left: 5px solid #8A2BE2;
    padding: 0.5em;
}

div.text_cell_render{
    font-size:13pt;
    }

</style>

In [3]:
import armageddon
import pandas as pd
import numpy as np
import folium
import random


# Damage Mapper


<div class="file">

## Locator.py

</div>

### Great_circle_distance

Calculate the great circle distance (in metres) between pairs of points specified as latitude and longitude on a spherical Earth.

```
def great_circle_distance(latlon1, latlon2):
```

with the inputs `latlon1` `latlon2` lists of 2-element arrays, with first entry to be latitude, second entry to be longitude.

The formulae we implemented is the spherical Vincenty formula

$$\frac{r}{R_p}=\arctan\frac{\sqrt{(\cos\varphi_2\sin|\lambda_1-\lambda_2|)^2+(\cos\varphi_1\sin\varphi_2-\sin\varphi_1\cos\varphi_2\cos|\lambda_1-\lambda_2|)^2}}{\sin\varphi_1 \sin\varphi_2+\cos\varphi_1\cos\varphi_2\cos|\lambda_1-\lambda_2|},$$

where $(\phi_1, \lambda_1)$ the location of the first point, and $(\phi_2, \lambda_2)$ the second.

Nothing much particularly need to be mentioned here, but do remind to convert between degrees and radians in calculation with trigonometric functions. We just used a factor of `math.pi/180` here, but commands like `np.degrees()`, `np.radians()` also helps.

```Python
    # turn degree to radian
    latlon1 *= math.pi / 180
    latlon2 *= math.pi / 180
```

return a 2-d array of great_circle_distances.




### Get_postcodes_by_radius

This function was designed to look up the postcodes inside some specific circle regions given a center point and a list of radius.

```
get_postcodes_by_radius(self, X, radii, sector=False):
```

This function requires three paramaters. X is a center point. radii contains a list of radius and the boolean value sector indicates whether postcode sectors are required or not.

In general, this function mainly used the Great_circle_distance function and dataframe operations from pandas to calculate as well as store distances between the center point and each postcode. First of all, the postcodes information was stored in a dataframe. Then, the difference of radius and distances between the center point and every postcode inside the postcode database were calculated and stored in a new column in the dataframe.


In [4]:
postcode_file='./resources/full_postcodes.csv'
postcodes_db = pd.read_csv(postcode_file).head(10)
location_latitude = np.array(postcodes_db['Latitude'])
location_longitude = np.array(postcodes_db['Longitude'])
locations = np.vstack((location_latitude, location_longitude))
locations = locations.T
distance = np.array([0,1,2,9,5,5,6,4,8,3])
radii = [2]

for r in radii:
    postcodes_db['distance_center_less'] = distance - r
    print(postcodes_db)

  Postcode   LAU218CD   Latitude  Longitude  distance_center_less
0  AB101AB  S31000932  57.149606  -2.096916                    -2
1  AB101AF  S31000932  57.148707  -2.097806                    -1
2  AB101AG  S31000932  57.149051  -2.097004                     0
3  AB101AH  S31000932  57.148080  -2.094664                     7
4  AB101AL  S31000932  57.150058  -2.095916                     3
5  AB101AN  S31000932  57.149700  -2.094742                     3
6  AB101AP  S31000932  57.148978  -2.095691                     4
7  AB101AQ  S31000932  57.148080  -2.094664                     2
8  AB101AR  S31000932  57.148080  -2.094664                     6
9  AB101AS  S31000932  57.148099  -2.097233                     1


Finally, we used the functionalities of dataframe to select postcodes inside the region according to the distance. 
```Python
    impacts = self.postcodes_db.loc[self.postcodes_db.distance_center_less < 0]
```
This function returned a list of lists that contains postcodes inside the region.

### Get_population_of_postcode

This function was used to look up the population according to a list of postcodes or postcodes sectors.
```
get_population_of_postcode(self, postcodes, sector=False):
```

For this function, only a list of lists containing postcodes and a boolean value indicating the type of results were required. Similarly, this function mainly used functionalities from pandas to achieve our goal. To lookup the population correctly, the postcode database and census database were firstly cleaned by removing spaces inside each postcode. The spaces inside input postcodes were also removed. This can help to reduce the influence of postcodes storing format.

```Python
self.census["geography_remove"] = self.census["geography"].apply(lambda x: x.replace(" ", ""))
self.postcodes_db["sector"] = self.postcodes_db["Postcode"].apply(lambda x: x[0:-2].replace(" ", ""))
```

For postcodes sectors, we can directly lookup them in the census database to find their populations. However, for postcodes, we had to approximate their populations according to their postcodes sector. To approximate that, we choose to calculate the average population inside the sector region. 

$$\frac{population \; of \; the \; sector}{number \; of \; postcodes \; in \; the \; sector}=approximated \; postcode \;  population,$$

Therefore, we both need to look up two databases. This can incur a large amount of time. Because some postcodes had the same sectors, to speed up the routine, duplications were firstly dropped when looking up and were recovered before return the result.

This function returns a list of lists contains corresponding populations.

<div class="file">

## Damage.py

</div>

### Damage_zones

To evaluate the impact of the Airblast (Crater) Event, we need to determine some criteria. The main cause of damage close to the impact site is a strong (pressure) blastwave in the air, known as the airblast. 

The zero surface locations can be derived from the following two formulaes:

$$\tag{1} \sin \varphi_2 = \sin \varphi_1\cos \left(\frac{r}{R_p}\right) +\cos \varphi_1\sin\left(\frac{r}{R_p}\right)\cos \beta,$$

$$ \tag{2} \tan(\lambda_2-\lambda_1) = \frac{\sin\beta\sin\left(\frac{r}{R_p}\right)\cos\varphi_1}{\cos\left(\frac{r}{R_p}\right)-\sin\varphi_1\sin\varphi_2}.$$

given $(\varphi_1,\lambda_1)$ the meteoroid entry point (latitude and longitude), $r$ from the  `outcome['burst_distance']`, we can get $(\varphi_2,\lambda_2)$ the surface zero point by direct computation.

Empirical data suggest that the following equation holds: 

\begin{equation*}
p(r) = 3.14 \times 10^{11} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-1.3} + 1.8 \times 10^{7} \left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)^{-0.565}
\end{equation*}

with $p$ the pressure (in Pa), explosion energy $E_k$ (in kilotons of TNT equivalent), burst altitude $z_b$ (in m) and horizontal range $r$ (in m).




Now if we treat the $\left(\frac{r^2 + z_b^2}{E_k^{2/3}}\right)$ as a single variable $X$, it would became a nice univariable non-linear equation: 

\begin{equation*}
f(X):= \, 3.14 \times 10^{11} \, X^{-1.3} + 1.8 \times 10^{7} \, X^{-0.565} - p(r) = 0  
\end{equation*}


This allows us to solve for $X$ given known pressure levels. And further to solve the blast radii $r$ s with known $E_k$, $z_b$. They comes from our solver's output `outcome['burst_energy']` and `outcome['burst_altitude']`.


<br>

```Python
def damage_zones(outcome, lat, lon, bearing, pressures):
    ...
    return blat, blon, damrad
```


### 2nd order Newton-Raphson Method to solve the non-linear equation

For non-linear equation $f(x) = 0$, suppose it has a unique solution $C$ over $(0, +\infty)$  (note equation (1) does have a unique one because it has monotonly increasing smooth 1st derivative)

Then we consider the iteration function:

\begin{equation}
\phi(x):= x + f(x) \, \left[ u(x) + f(x) \, w(x) \right]
\end{equation}

suppose that $\phi(C) = C$.

And by Taylor expansion we have:
\begin{align}
\phi(x)  &= \phi(C) + (x-C) \, \phi'(C) + \frac{(x-C)^2}{2!} \, \phi''(C) + \frac{(x-C)^3}{3!} \, \phi'''(C) + \cdots + \frac{(x-C)^n}{n!} \, \phi^{n}(C) \cdots\\
x_{i+1} - C &= (x-C) \, \phi'(C) + \frac{(x -C)^2}{2!} \, \phi''(C) + \frac{(x-C)^3}{3!} \, \phi'''(C) + \cdots\\
\left| x_{i+1} - C \right | &= \left| x_i -C \right| \, \left| \phi'(C) + \frac{(x_i -C)}{2!} \, \phi''(C) + \frac{(x_i -C)^2}{3!} \, \phi'''(C) + \cdots \right|
\end{align}


And recall 1st order Newton, we set $\phi'(C) = 0$, thus we have quadratic convergence:

$$
\left| x_{i+1} - C \right | = \left| x_i -C \right|^2 \, \left|\frac{1}{2!} \, \phi''(C) + \frac{(x_i-C)}{3!} \, \phi'''(C) + \cdots \right|
$$

And for 2nd order Newton, we set both $\phi'(C) = 0$ and $\phi''(C) = 0$, thus we have cubic convergence:

$$
\left| x_{i+1} - C \right | = \left| x_i -C \right|^3 \, \left|\frac{1}{3!} \, \phi'''(C) + \frac{(x_i-C)}{4!} \, \phi^{(4)}(C) + \cdots \right|
$$

Thus the computation to define $u(x)$ , $w(x)$ are as follows:

$\phi'(C) = 0$ :

\begin{align}
\phi'(x) &= 1 + f'(x)  \, \left[ u(x) + f(x) \, w(x) \right] + f(x) \, \left[ u'(x) + f'(x) \, w(x) + f(x) \, w'(x) \right]\\
\\
0 = \phi'(C) &= 1 + f'(C) \, u(C) 
\end{align}

dont't know $C$, we can just choose $$u(x) = - \frac{1}{f'(x)}$$

$\phi''(C) = 0$ :

\begin{align}
\phi''(x) &= f''(x)  \, \left[ u(x) + f(x) \, w(x) \right] + f'(x)  \, \left[ u'(x) + f'(x) \, w(x) + f(x) \, w'(x) \right] \\
\\
&+ f'(x) \, \left[ u'(x) + f'(x) \, w(x) + f(x) \, w'(x) \right] + f(x) \, \left[ u''(x) + f''(x) \, w(x) + 2 \, f'(x) \, w'(x) + f(x) \, w''(x) \right]\\
\\
\phi''(C) &= 0 = f''(C) \, u(C) + f'(C) \, u'(C) +  \left[f'(C)\right]^2 \, w(C) + f'(C) \, u'(C) + \left[f'(C)\right]^2 \, w(C)\\
\\
0 &= f''(C) \, \left(- \frac{1}{f'(C)}\right) + 2 \,\left[ f'(C) \, \left(\frac{f''(C)}{\left[f'(C)\right]^2}\right) + \left[f'(C)\right]^2 \, w(C) \right]\\ 
\\
0 &= \frac{f''(C)}{f'(C)} + 2 \, \left[f'(C)\right]^2 \, w(C)
\end{align}

dont't know $C$, we can just choose $$w(x) = - \frac{f''(x)}{2 \, \left[f'(x)\right]^3}$$

Thus, we could implement our algorithm 

```Python
def second_order_Newton(f, dfdx, df2dx2, initial_guess):
'''
Parameters
    ----------

    f: function
        the non-linear function we need to solve, (eg. f(x) = 0)
    dfdx: function
        the 1st derivative for function f
    df2dx2: function
        the 2nd derivative for function f
    intial_guess: float
        the start point of the iteration
        
    Returns
    -------

    root: float
        the numerical solution for the Newton method    
'''
    
```

The choice of initial guess for Newton Method is triky, here we choose to start from each $p(r)$ given, and we have checked that, for the pressure input $\left(10^{-2} , 10^6\right)$, Newton is reliable.  

## (Extension Functionality, combined with given specified scenario)

Define parameters for the Asteroid  

In [5]:
# define the input parameters
asteroid = {'radius': 20, 'angle': 45, 'strength': 1e7,
                  'density': 8000, 'velocity': 17e3,
                  'lat': 53.79, 'lon': -3.55, 'bearing': 112.}

Import the solver, assuming the atmosphere density to be ''exponential''

In [6]:
# create an instance
earth = armageddon.Planet(atmos_func='exponential')

# Solve the atmospheric entry problem
result = earth.solve_atmospheric_entry(radius=asteroid['radius'], angle=asteroid['angle'],
                                       strength=asteroid['strength'], density=asteroid['density'],
                                       velocity=asteroid['velocity'])
# as a column to the result dataframe
result = earth.calculate_energy(result)

# Determine the outcomes of the impact event
outcome = earth.analyse_outcome(result)
print(outcome)

{'outcome': 'Airburst', 'burst_energy': 6233.650011863119, 'burst_peak_dedz': 1131.8068466463383, 'burst_altitude': 9243.425840745811, 'burst_distance': 90996.41771354491}


To drive to the plotting part, let's first use our `damage_zone` to process an example, which returns us the surface_zero point and blast radiis for 4 Damage Levels. 

And define pressure levels, according to:

|  Damage Level |  Description    | Pressure (kPa) |
|:-------------:|:---------------:|:--------------:|
|  1  |  ~10% glass windows shatter    |     1.0      |
|  2  | ~90% glass windows shatter     |     3.5      |
|  3  | Wood frame buildings collapse  |     27      |
|  4  | Multistory brick buildings collapse  |     43      |


In [7]:
# Calculate the blast location and damage radius for several pressure levels
blast_lat, blast_lon, damage_rad = armageddon.damage_zones(outcome,
                                                           lat=asteroid['lat'], lon=asteroid['lon'], bearing=asteroid['bearing'],
                                                           pressures=[1e3, 3.5e3, 27e3, 43e3])

In [8]:
print('surface zero point:', '(', blast_lat, blast_lon,')')
print('damage radii: ', damage_rad)

surface zero point: ( 53.47663774575009 -2.27503151372165 )
damage radii:  [111457.13034072031, 40691.54501499831, 7659.155070316064, 2307.2172269548146]


<div class="file">

## Mapping.py

</div>

Let's plot circles for these data:

In [9]:
plot = armageddon.plot_circle(blast_lat, blast_lon, damage_rad , map=None)

As well as a line from the entry point to the surface zero point.

In [10]:
plot = armageddon.plot_polyline(asteroid['lat'], asteroid['lon'], blast_lat, blast_lon, map = plot)

save the plot:

In [11]:
plot.save('damage_map_senario.html')

In [12]:
from IPython.display import IFrame
IFrame(src='damage_map_senario.html', width=700, height=600)

Find out the postcode units affected, with their population

In [13]:
# The PostcodeLocator tool
locator = armageddon.PostcodeLocator()

# Find the postcodes in the damage radii
postcodes = locator.get_postcodes_by_radius((blast_lat, blast_lon), radii=damage_rad)

# Find the population in each postcode
population = locator.get_population_of_postcode(postcodes)



Postcode sectors affected

In [ ]:
# Alternatively find the postcode sectors in the damage radii, and populations of the sectors
sectors = locator.get_postcodes_by_radius((blast_lat, blast_lon), radii=damage_rad, sector=True)
population_sector = locator.get_population_of_postcode(sectors, sector=True)

Level 1 (~10% glass windows shatter)

In [61]:
postcode_sec_df = pd.DataFrame(data = population_sector[0], index = sectors[0], columns = ['population'] )
postcode_sec_df[postcode_sec_df['population']!= 0]

,population
HD2 1,15473
SK141,5911
L21 1,3682
CH449,3820
DN161,9044
...,...
WV3 9,8097
LN1 2,11484
FY4 3,10358
LE125,7482


In [62]:
print('a total number of', sum(postcode_sec_df.population), 'people is affected by this event.')

a total number of 17029776 people is affected by this event.


Level 2 (~90% glass windows shatter)

In [65]:
postcode_sec_df_2 = pd.DataFrame(data = population_sector[1], index = sectors[1], columns = ['population'] )
postcode_sec_df_2[postcode_sec_df_2['population']!= 0]

,population
HD3 3,15131
HD2 1,15473
SK141,5911
WN7 2,10026
BL7 0,3180
...,...
WN7 4,7214
L36 6,2522
BL0 0,5575
SK8 4,8557


In [66]:
print('a total number of', sum(postcode_sec_df_2.population), 'people suffered from level 2 damage.')

a total number of 5136843 people suffered from level 2 damage.


Level 3 (Wood frame buildings collapse)

In [67]:
postcode_sec_df_3 = pd.DataFrame(data = population_sector[2], index = sectors[2], columns = ['population'] )
postcode_sec_df_3[postcode_sec_df_3['population']!= 0]

,population
M16 7,7628
M30 8,9115
M15 6,7710
M8 9,3513
M18 7,12643
...,...
M5 4,5638
M14 6,19997
M11 1,5759
M40 9,4579


In [68]:
print('a total number of', sum(postcode_sec_df_3.population), 'people suffered from level 3 damage.')

a total number of 918891 people suffered from level 3 damage.


Level 4 (Multistory brick buildings collapse)

In [69]:
postcode_sec_df_4 = pd.DataFrame(data = population_sector[3], index = sectors[3], columns = ['population'] )
postcode_sec_df_4[postcode_sec_df_4['population']!= 0]

,population
M16 7,7628
M15 6,7710
M50 1,942
M3 4,2266
M3 7,2992
M6 5,8289
M3 3,544
M15 4,8271
M5 3,5177
M1 4,215


In [72]:
postcode_unit_df_4 = pd.DataFrame(data = population[3], index = postcodes[3], columns = ['population'] )
postcode_unit_df_4[postcode_unit_df_4['population']!= 0]


,population
M1 1AD,9.385542
M1 1BZ,9.385542
M1 1PL,9.385542
M1 1PT,9.385542
M1 1PW,9.385542
...,...
M7 2JS,31.209302
M7 2JT,31.209302
M7 2JU,31.209302
M7 2LD,31.209302


In [73]:
print('a total number of', sum(postcode_sec_df_4.population), 'people suffered from level 4 damage.')

a total number of 146803 people suffered from level 4 damage.


### Uncertainty analysis (impact_risk)

The function `impact_risk` in **damage.py** is written  tot take an additional set of inputs, describing the standard deviation of each input parameter, as well as the nominal input parameters. 

In [22]:
# define the standard deviations
as_stdevs = {'radius': 1, 'angle': 1, 'strength': 5e5,
                   'density': 500, 'velocity': 1e3,
                   'lat': 0.025, 'lon': 0.025, 'bearing': 0.5}


The functions looks like
```Python
def impact_risk(planet, means=fiducial_means, stdevs=fiducial_stdevs,
                pressure=27.e3, nsamples=100, sector=True):
```

We will determine lists of surface_zero coordinates and radiis to define circles, and use `get_postcodes_by_radius` and `get_population_of_postcode` functions to find out the postcodes sector (unit) inside each circle, as well as their population.

We will accumulate the time of each postcodes sector (unit) appears in total loops, the risk value is calculated by :

<br>

$$
risk := \frac{number \, of \,  time \, affected }{n \, sample} \, \cdot population
$$